# DART공시데이터 활용요소

### 적시성
* 과거매출액 추정치, 실제잠정실적공시  비교
* 사업목적수( 임시주주총회 등)
* 주식보유율 5%초과공시 - 대량보유상황보고서
* 인수공시

### 조회성
* 배당율
* 단일판매공급계약
* 조직구조


### 배당율 데이터 수집

In [1]:
#  초기설정
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3
import requests
import re

cnx = sqlite3.connect('financedata.db')
cnx.execute('ATTACH DATABASE "dart.db" AS "dart"')

#---------------------------------------------------------------------------------

# 0) 프레임 url호출
def get_report_doc_urls(rcp_no):
    '''접수번호(rcp_no)에 해당하는 모든 하위 문서 URL을 추출
    '''
    doc_urls = []
    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=%s" % (rcp_no)
    r = requests.get(url)
    reg = re.compile('viewDoc\((.*)\);')
    params = []
    matches = reg.findall(r.text)
    for m in matches: 
        params.append(m.replace("'", "").replace(" ", "").split(","))
   
    doc_url_tmpl = "http://dart.fss.or.kr/report/viewer.do?rcpNo=%s&dcmNo=%s&eleId=%s&offset=%s&length=%s&dtd=%s" 

    for p in params:
        if rcp_no == p[0]:
            doc_urls.append( doc_url_tmpl % tuple(p) )
        
    return doc_urls

#-----------------------------------------------------------------------------------

# 1) 2000년 이후 현금 현물 배당결정공시 추출
query = """
    select * 
    from stock_dart
    where 종목코드= '005380' and 보고서명 like '현금ㆍ현물배당결정%' and 접수날짜 between '2000-10-01' and '2018-12-31'
    """

#df_dart = pd.read_sql(query, cnx, parse_dates=['접수날짜'], index_col=['접수날짜'])
df_dart = pd.read_sql(query, cnx, parse_dates=['접수날짜'])


# 2) 공시 보고서 링크 추출
result = []

for ix, row in df_dart.iterrows():
    print(ix, 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + row['접수번호'],  row['보고서명'])
    doc_urls = get_report_doc_urls(row['접수번호'])

    for url in doc_urls:  
        #print (ix,' | ', url)
        r = requests.get(url)
        #print(r.text[:20])
        soup = BeautifulSoup(r.text, 'lxml')
        trs = soup.find_all('tr') # table은 항상 1개
        
        #print(result[:200])
        
        values ={}
        search_re = {
            '배당기준일':'배당기준일',
            '구분' :'배당구분',
            '보통주배당율': '보통주'    

        }
        values = {}
        for tr in trs:
            tds = tr.find_all('td')
            left_text = ' '.join([td.text.strip() for td in tds[:-1]])
            right_text = tds[-1].text.strip()
                        
            for key in search_re.keys():
                if re.search(search_re[key], left_text):
                    values[key] = right_text            
            
                    
        if values:
            result.append(values)
            
#---------------------------------------------------------------------------
# 3) df으로 데이터 추출 & 기존 데이터프레임 병합
df = pd.DataFrame(result)
df['보통주배당율'] = df['보통주배당율'].str.replace('%','')
df['보통주배당율'] = df['보통주배당율'].astype(float)
#df.set_index('배당기준일', inplace=True)
#df

df_dart2 = df_dart[['접수날짜','종목명','종목코드']]
df_dart2['구분'] = df['구분']
df_dart2['배당기준일'] = df['배당기준일']
df_dart2['보통주배당율'] = df['보통주배당율']
#df_dart2

0 http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20100128800184 현금ㆍ현물배당결정


ConnectionError: HTTPConnectionPool(host='dart.fss.or.kr', port=80): Max retries exceeded with url: /dsaf001/main.do?rcpNo=20100128800184 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f14b5adf7b8>: Failed to establish a new connection: [Errno 110] Connection timed out',))

### 생성 데이터프레임 확인

In [ ]:
df_dart2

## 신규 DB(new_dart.db) & 테이블(dart_final) 생성

In [10]:
#  파일이 존재하면 삭제 (사용 주의 )
import os

fname = 'new_dart.db'
if os.path.isfile(fname):
    os.unlink(fname)
    
#  파일이 존재하지 않으면 생성 된다.
db = sqlite3.connect(fname)   

# stock_master 테이블  쓰기
import sqlite3
conn = sqlite3.connect('new_dart.db')

df_dart2.to_sql('dart_final', conn, if_exists='replace')

### 데이터 확인

In [18]:
# 테이블  읽기
import pandas as pd
import sqlite3

conn = sqlite3.connect('new_dart.db')

df = pd.read_sql('SELECT * FROM dart_final  order by 1', conn)
df.head()


cnx = sqlite3.connect('new_dart.db')
cnx.execute('ATTACH DATABASE "new_dart.db" AS new_dart;')
cnx1 = sqlite3.connect('financedata.db')
cnx1.execute('ATTACH DATABASE "dart.db" AS stock_dart;')

code = '005380'
query = "SELECT * FROM dart_final where 종목코드='%s' order by 1" % code
pd.read_sql(query, conn, parse_dates=['접수날짜'])


# query = "select * from stock_desc where 종목코드='%s'" % code
# pd.read_sql(query, cnx1)

# query = "select * from stock_desc where 종목코드='%s'" % code
# pd.read_sql(query, cnx1)

# the_day = '2017-01-25'.date()
# start = the_day - timedelta(days=30)
# end = the_day + timedelta(days=60)
# sql_tmpl = "select * from stock_price where 종목코드='%s' and 날짜 between '{start}' and '{end}'" % code
# sql = sql_tmpl.format(start=start, end=end)
# return pd.read_sql(sql, cnx, parse_dates=['날짜'], index_col=['날짜'])

,index,접수날짜,종목명,종목코드,구분,배당기준일,보통주배당율
0,0,2010-01-28,현대자동차,005380,결산배당,2009-12-31,1.0
1,1,2011-01-27,현대자동차,005380,결산배당,2010-12-31,0.8
2,2,2012-01-26,현대자동차,005380,결산배당,2011-12-31,0.8
3,3,2013-01-24,현대자동차,005380,결산배당,2012-12-31,0.9
4,4,2014-01-23,현대자동차,005380,결산배당,2013-12-31,0.9
5,5,2015-01-22,현대자동차,005380,결산배당,2014-12-31,1.7
6,6,2015-07-23,현대자동차,005380,중간배당,2015-06-30,0.8
7,7,2016-01-26,현대자동차,005380,결산배당,2015-12-31,2.0
8,8,2016-07-26,현대자동차,005380,중간배당,2016-06-30,0.7
9,9,2017-01-25,현대자동차,005380,결산배당,2016-12-31,2.1


## 챗봇 서버

In [16]:
# telegram DART Monitor 04
# 보고서 생성기 (sales_port_gen.py)

import logging
import re
import sqlite3
import pandas as pd
from telegram.ext import Updater
from telegram.ext import CommandHandler
from telegram.ext import MessageHandler, Filters
import subprocess

# logging 설정
log_format = '%(asctime)s - %(levelname)s - %(message)s'
logging.basicConfig(format=log_format, filename='./telegram.log', level=logging.INFO)


# Updater 생성 - https://telegram.me/pyfinance_bot
# @pyfinance_bot:
TOKEN = '408675446:AAGfv0si1CLbMb2i9R1Mu8CtQIicRwogEa0'

# 사용자에 대한 설정 (1명만)
user = {}
user['chat_id'] = "413342879"  # 사용자의 chat_id 고유번호

# 텔레그램 챗봇 서버 토큰
print('텔레그램 챗봇 서버를 시작합니다')
updater = Updater(token=TOKEN)

# Dispatcher 얻기
dispatcher = updater.dispatcher

def reply_message(bot, update):
    in_text = update.message.text
    print(in_text)
    cmd_list = in_text.split(' ')
    if '배당' in cmd_list[0]:
        종목코드, 종목명 = cmd_list[1], cmd_list[2]
        subprocess.call(['python3', 'bd_port_gen.py', 종목코드, 종목명])
        report_text = 종목명 +': ' + 'http://steweb.run.goorm.io/' + 종목코드 + '_' + 종목명 + '.html'
        bot.send_message(chat_id=update.message.chat_id, text=report_text)
    print(report_text)

# 메시지 핸들러 지정
message_handler = MessageHandler(Filters.text, reply_message)
dispatcher.add_handler(message_handler)

# 실행
updater.start_polling()

-f /root/.local/share/jupyter/runtime/kernel-2f76e637-f3ce-4b86-8e03-26996ad22bcf.json


IndexError: single positional indexer is out-of-bounds

## 배당율 보고서 생성기 

In [20]:
import sys
import os
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
from io import BytesIO
import base64
import re
import requests
from bs4 import BeautifulSoup
from jinja2 import Template
import time

cnx = sqlite3.connect('new_dart.db')
cnx.execute('ATTACH DATABASE "new_dart.db" AS new_dart;')

cnx1 = sqlite3.connect('financedata.db')
cnx1.execute('ATTACH DATABASE "dart.db" AS stock_dart;')

def get_corp_desc(code):
    query = "select 기업개요 from stock_desc where 종목코드='%s'" % code
    return pd.read_sql(query, cnx1)

def get_corp_price(code, the_day):
    start = the_day - timedelta(days=30)
    end = the_day + timedelta(days=60)
    sql_tmpl = "select * from stock_price where 종목코드='%s' and 날짜 between '{start}' and '{end}'" % code
    sql = sql_tmpl.format(start=start, end=end)
    return pd.read_sql(sql, cnx1, parse_dates=['날짜'], index_col=['날짜'])

def get_dart_sales(code):
    query = "select * from stock_dart where 종목코드='%s' and 보고서명 like '단일판매%%' and 접수날짜 >= '2010-01-01'" % code
    return pd.read_sql(query, cnx1, parse_dates=['접수날짜'])

def get_dart_finals(code):
    query = "SELECT * FROM dart_final where 종목코드='%s' order by 1" % code
    return pd.read_sql(query, cnx, parse_dates=['접수날짜'])

def get_report_doc_urls(doc_id):
    doc_urls = []
    url = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=%s" % (doc_id)
    time.sleep(1)
    r = requests.get(url)

    reg = re.compile('viewDoc\((.*)\);')
    params = []
    matches = reg.findall(r.text)
    for m in matches: 
        params.append(m.replace("'", "").replace(" ", "").split(","))
   
    doc_url_tmpl = "http://dart.fss.or.kr/report/viewer.do?rcpNo=%s&dcmNo=%s&eleId=%s&offset=%s&length=%s&dtd=%s" 

    for p in params:
        if doc_id == p[0]:
            doc_urls.append( doc_url_tmpl % tuple(p) )
        
    return doc_urls

if __name__ == "__main__":
    html_report_tmpl = """
    <!DOCTYPE html>
    <html>
    <head lang="ko">
        <style>
            img { display: block; }
        </style>
        <meta charset="UTF-8">
        <title>{{ report_data.title }}</title>
        <style>  
            table { border-collapse:collapse; }  
            table th, table td { border:2px solid gray; }
        </style>
    </head>
    <body>
        <h1>{{report_data.corp_name}}({{report_data.code}}) - {{ report_data.title }}</h1>
        <h2>개요</h2>
        {{ report_data.corp_desc }}
        <br>
        <h2>배당율 리스트</h2>
        <table>
        <thead>
        <tr>
        <th style="width:200px;">접수날짜</th>
        <th style="width:200px;">구분</th>
        <th style="width:200px;">배당기준일</th>
        <th style="width:200px;">보통주배당율</th>
        <tr>
        </thead>
        
        {% for event in event_list %}
        <tbody>
        <tr>
        <td align=center>{{ event.date }}</td>
        <td align=center>{{ event.gb }}</td>
        <td align=center>{{ event.bdday }}</td>
        <td align=center>{{ event.bdrate }}%</td>
        </tr>
        </tbody>
        {% endfor %}
    </body>
    </html>
    """
	
    종목코드, 종목명 = sys.argv[1], sys.argv[2]
    print(종목코드, 종목명)
	
    report_data = {}
    report_data['title'] = '종목정보 및 배당율 조회'
    report_data['code'] = 종목코드
    report_data['corp_name'] = 종목명
    report_data['corp_desc'] = get_corp_desc(종목코드).iloc[0,0]

    filename = 종목코드 + '_' + 종목명 + '.html'
    print (filename)

    event_list = []
    df_dart = get_dart_sales(종목코드);
    df_dart_final = get_dart_finals(종목코드);

    for ix, row in df_dart_final.iterrows():
        event = {}
        the_day = row['접수날짜'].date() # 공시일
        event['date'] = the_day.strftime("%Y-%m-%d")
        event['gb'] = row['구분']
        event['bdday'] = row['배당기준일']
        event['bdrate'] = row['보통주배당율']        
        event_list.append(event)

    t = Template(html_report_tmpl)
    html_report = t.render(report_data=report_data, event_list=event_list)

    with open(filename, "w") as f:
        f.write(html_report)

-f /root/.local/share/jupyter/runtime/kernel-2f76e637-f3ce-4b86-8e03-26996ad22bcf.json


IndexError: single positional indexer is out-of-bounds

# 웹서버 실행

python3 -m http.server 8888


# 챗봇 실행

python3 telegram_dart_mon_bd.py